In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import numpy as np
from citipy import citipy
# Import the API key.
from config import weather_api_key


In [2]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

755

In [6]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | port keats
Processing Record 2 of Set 1 | tuatapere
Processing Record 3 of Set 1 | ushuaia
Processing Record 4 of Set 1 | maun
Processing Record 5 of Set 1 | kuytun
Processing Record 6 of Set 1 | faanui
Processing Record 7 of Set 1 | bredasdorp
Processing Record 8 of Set 1 | upernavik
Processing Record 9 of Set 1 | barrow
Processing Record 10 of Set 1 | lebu
Processing Record 11 of Set 1 | longyearbyen
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | rikitea
Processing Record 14 of Set 1 | torbay
Processing Record 15 of Set 1 | saint anthony
Processing Record 16 of Set 1 | albany
Processing Record 17 of Set 1 | kruisfontein
Processing Record 18 of Set 1 | port lincoln
Processing Record 19 of Set 1 | vaini
Processing Record 20 of Set 1 | punta arenas
Processing Record 21 of Set 1 | saint-augustin
Processing Record 22 of Set 1 | poum
Processing Record 23 of Set 1 | norman w

In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Port Keats,-14.2500,129.5500,88.92,53,3,7.74,AU,clear sky
1,Tuatapere,-46.1333,167.6833,53.82,65,90,10.42,NZ,overcast clouds
2,Ushuaia,-54.8000,-68.3000,42.46,56,20,18.41,AR,few clouds
3,Maun,-19.9833,23.4167,80.56,36,0,5.75,BW,clear sky
4,Kuytun,54.3419,101.5036,34.63,71,83,6.93,RU,broken clouds
5,Faanui,-16.4833,-151.7500,79.18,73,0,17.60,PF,clear sky
6,Bredasdorp,-34.5322,20.0403,58.32,94,100,11.50,ZA,light rain
7,Upernavik,72.7868,-56.1549,32.61,71,100,14.36,GL,overcast clouds
8,Barrow,71.2906,-156.7887,30.22,74,90,10.36,US,haze
9,Lebu,-37.6167,-73.6500,50.16,85,100,5.95,CL,overcast clouds


In [9]:
# Reorder the columns
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Port Keats,AU,-14.2500,129.5500,88.92,53,3,7.74,clear sky
1,Tuatapere,NZ,-46.1333,167.6833,53.82,65,90,10.42,overcast clouds
2,Ushuaia,AR,-54.8000,-68.3000,42.46,56,20,18.41,few clouds
3,Maun,BW,-19.9833,23.4167,80.56,36,0,5.75,clear sky
4,Kuytun,RU,54.3419,101.5036,34.63,71,83,6.93,broken clouds
5,Faanui,PF,-16.4833,-151.7500,79.18,73,0,17.60,clear sky
6,Bredasdorp,ZA,-34.5322,20.0403,58.32,94,100,11.50,light rain
7,Upernavik,GL,72.7868,-56.1549,32.61,71,100,14.36,overcast clouds
8,Barrow,US,71.2906,-156.7887,30.22,74,90,10.36,haze
9,Lebu,CL,-37.6167,-73.6500,50.16,85,100,5.95,overcast clouds


In [11]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")